In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ratings = pd.read_table('./app/datasets/ml-1m/ratings.dat', delimiter='::', names=['userId', 'movieId', 'rating', 'timestamp'], engine='python')

In [3]:
df_movies = pd.read_csv('./app/datasets/movies_imdb.csv')
df_movies = df_movies.dropna(subset=['genres', 'cast','directors', 'runtimes', 'title', 'year'])
df_movies = df_movies.loc[:, ['movieId', 'genres','cast', 'directors', 'runtimes', 'title', 'year']]

In [4]:
df_ratings = df_ratings[df_ratings['movieId'].isin(df_movies['movieId'])]

In [5]:
def get_weights(df_movies):
    def add_in_dict(d, l):
        for e in l:
            n = d.get(e, 0)
            d[e] = n+1

    genres = {}
    cast = {}
    directors = {}
    for index, movie in df_movies.iterrows():
        movie_genres = movie['genres'].split('|')
        add_in_dict(genres, movie_genres)
        movie_cast = movie['cast'].split('|')
        add_in_dict(cast, movie_cast)
        movie_directors = movie['directors'].split('|')
        add_in_dict(directors, movie_directors)
    genres = pd.Series(genres, name='genres')
    cast = pd.Series(cast, name='cast')
    directors = pd.Series(directors, name='directors')
    years = df_movies['year'].value_counts()
    return genres, cast, directors, years

In [6]:
import json
def split(df_ratings, user):
        df_r_movies = df_ratings[df_ratings['userId'] == user]
        df_r_movies = df_r_movies.set_index('movieId')
        data = {}
        with open('./app/datasets/index.txt') as json_file:
            data = json.load(json_file)
        user_index = data[str(user)]
        train = df_r_movies.loc[user_index['train'], :]
        test = df_r_movies.loc[user_index['test'], :]
        return train, test
train, test = split(df_ratings, 6040)

In [7]:
train

,userId,rating,timestamp
movieId,,,
2391,6040,4,956705112
246,6040,5,956704613
2303,6040,5,960971857
3334,6040,5,960971875
535,6040,4,964828734
...,...,...,...
913,6040,5,956716845
1300,6040,5,956715421
2076,6040,5,956715676


In [8]:
genres, cast, directors, years = get_weights(df_movies)

In [9]:
def get_gcdy_preference(train, df_movies):
    def add_in_list(d, l):
        for e in l:
            if e not in d:
                d.append(e)
            
    df = df_movies.loc[train[train.rating>=3].index, :]
    genres = []
    cast = []
    directors = []
    for index, movie in df.iterrows():
        try:
            movie_genres = movie['genres'].split('|')
            add_in_list(genres, movie_genres)
            movie_cast = movie['cast'].split('|')
            add_in_list(cast, movie_cast)
            movie_directors = movie['directors'].split('|')
            add_in_list(directors, movie_directors)  
        except AttributeError:
            return [], [], [], []
    return genres, cast, directors, df.year.unique().tolist()



In [10]:
def get_wi(lista, preference, all_movies, total, inc):
    wi = 0.0
    for valor in lista:
        wi += all_movies[valor]/total
        if valor in preference:
            wi += inc
    return wi
def get_wa(lista, c, cast, total):
    return get_wi(lista, c, cast, total, 0.4)

def get_wd(lista, d, directors, total):
    return get_wi(lista, d, directors, total, 0.3)

def get_wg(self, lista, g, genres, total):
        return get_wg(lista, g, directors, total, 0.2)
    
def get_wy(lista, y, years, total):
    return get_wi(lista, y, years, total, 0.1)
        
def get_wr(movieId, df_ratings):
    ratings = df_ratings[df_ratings['movieId'] == movieId].rating.value_counts()
    wr = 0.0
    
    for index, count in ratings.iteritems():
        if index >=3:
            if count <= 100:
                wr += index
            elif count <= 1000:
                wr += index*2
            else:
                wr += index*3
        else:
            if count <= 100:
                wr += 1
            elif count <= 1000:
                wr += 2
            else:
                wr += 3
    return wr
        

In [11]:

def get_user_weights(user, train, test, df_movies):
    total = df_movies.shape[0]
   
    g, c, d, y = get_gcdy_preference(train, df_movies)
    ratings_test = df_ratings[df_ratings.userId == user][df_ratings.movieId.isin(test.index)]
    df_ratings_w_test = df_ratings[~df_ratings.index.isin(ratings_test.index)]
    weights = []
    for index, movie in df_movies.iterrows():
        wa = get_wa(movie['cast'].split('|'), c, cast, total)
        wd = get_wd(movie['directors'].split('|'), d, directors, total)
        wy = get_wy([movie['year']], y, years, total)
        wg = get_wd(movie['genres'].split('|'), g, genres, total)
        wr = get_wr(movie['movieId'], df_ratings_w_test)
        weights.append([wa, wd, wg, wy, wr])
    weights = pd.DataFrame(weights, index=df_movies.movieId, columns=['Wa', 'Wd','Wg', 'Wy', 'Wr'])
    return weights
    #weights = pd.DataFrame(weights, columns=['Wa', 'Wd', 'Wy', 'Wr'])
    
        

In [14]:
last_user = None
data = {}
with open('./app/datasets/index.txt') as json_file:
    data = json.load(json_file)
last_user = data.get('last_user',  None)
if last_user:
    del data['last_user']
else:
    last_user = -1
users = list(data.keys())
data['last_user']

0

In [15]:
weights = get_user_weights(users[0], train, test, df_movies)

/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


KeyboardInterrupt: 

In [ ]:
X = weights[weights.index.isin(test.index)]

In [18]:
from sklearn.cluster import KMeans
#kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

In [ ]:
X['cluster'] = kmeans.labels_

In [ ]:
X['W'] = X['Wa'] + X['Wd'] + X['Wy'] + X['Wr']

In [ ]:
recommender_cluster = X.groupby('cluster').mean()['W'].sort_values(ascending=False).index[0]

In [ ]:
movies = X[X.cluster == recommender_cluster].index.tolist()

In [ ]:
movies[]

In [16]:
def recommend(user):
    train, test = split(df_ratings, int(user))
    weights = get_user_weights(user, train, test, df_movies)
    X = weights[weights.index.isin(test.index)]
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    X['cluster'] = kmeans.labels_
    X['W'] = X['Wa'] + X['Wd'] + X['Wy'] + X['Wg'] + X['Wr']
    recommender_cluster = X.groupby('cluster').mean()['W'].sort_values(ascending=False).index[0]
    movies = X[X.cluster == recommender_cluster].index.tolist()
    if len(movies) > 15:
        return movies[:15]
    else:
        return movies

In [57]:
recommend(users[0])

/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying t

[7, 11, 34, 45, 52, 110, 141, 144, 163, 164, 173, 196, 198, 222, 231]

In [45]:
for index, movie in df.iterrows():
    if type(movie['cast']) ==  float:
        print(index, '-', movie['genres'])

3139 - nan
637 - nan
3668 - nan
3269 - nan
1287 - nan
3586 - nan
1387 - nan
708 - nan


In [27]:
train

,userId,rating,timestamp
movieId,,,
3197,29,3,978136239
3197,62,4,977966954
3197,117,4,977496477
3197,118,3,977501416
3197,123,3,977529627
...,...,...,...
2384,6015,2,956779281
2384,6016,5,956779117
2384,6035,4,956713027


In [24]:
df = df_movies.loc[train[train.rating>=3].index, :]
for index, movie in df.iterrows():
    print(index, '-', movie['genres'].split('|'))

3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']
3197 - ['Comedy', 'Mystery', 'Romance']


1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Drama', 'Mystery', 'Thriller']
1866 - ['Crime', 'Dr

1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comedy', 'Romance']
1285 - ['Comed

321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Adventure', 'Comedy']
321 - ['Ad

1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
1677 - ['Action', 'Adventure', 'Comedy', 'Family']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
554 - ['Comedy', 'Drama']
3517 - ['Adventure', 'Comedy', 'Family']
3517 - ['Adventure', 'Comedy', 'Family']
2321 - ['Action', 'Adventure', 'Comedy', 'Romance']
2321 - ['Action', 'Adventure', 'Comedy', 'Romance']
2321 - ['Action', 'Adventure', 'Comedy', '

...

805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Adventure', 'Drama']
805 - ['Advent

1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 'Fantasy', 'Music']
1254 - ['Animation', 'Family', 

903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Drama', 'History', 'Romance', 'War']
903 - ['Dr

1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'Horror']
1189 - ['Comedy', 'H

AttributeError: 'float' object has no attribute 'split'

In [19]:
last_user = -1
n = len(users)
for i in range(n -215, n):
    print('faltam', n - i, 'users')
    user = users[i]
    data = {}
    if last_user >= 0:
        with open('./app/recomendacoes/cb-recomendacoes.txt') as json_file:
            data = json.load(json_file)
    data[str(user)] = recommend(int(user))
    last_user = i
    with open('./app/recomendacoes/cb-recomendacoes.txt', 'w') as outfile:
        json.dump(data, outfile)

faltam 215 users


/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/math

faltam 214 users


/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future

faltam 213 users


...

faltam 3 users


/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


faltam 2 users
faltam 1 users


/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/matheus/eclipse-workspace/RS-MOP/rs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


ValueError: invalid literal for int() with base 10: 'last_user'

In [55]:
df_movies.isna().all()

movieId      False
genres       False
cast         False
directors    False
runtimes     False
title        False
year         False
dtype: bool